# Example use of the Sonic module Evo.jl

See https://julialang.org, https://juliadb.org and https://juliastats.org for reference.

In [35]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

Activating environment at `~/Documents/Calendar/Work/Noda/Code/Sonic.jl/Project.toml`


In [36]:
import Dates, JuliaDB, Random, StatsPlots

In [37]:
using StatsPlots
ENV["GKSwstype"] = "100"
gr(fmt = :png)

Plots.GRBackend()

Get secret token,

In [38]:
include("../src/EvoSecrets.jl")
secret = EvoSecrets.veab

(source = (endpoint = "https://evo.elvaco.se", username = "jens.brage@noda.se", password = "bV6CDHnC&tMm"), target = "db/evo/veab")

Get access token,

In [55]:
include("../src/Evo.jl")
access = Evo.get_access(secret)

(source = (endpoint = "https://evo.elvaco.se", username = "jens.brage@noda.se", password = "bV6CDHnC&tMm"), target = "db/evo/veab", bearer = "40415937-4759-42d3-ba5f-78cb2e221431")

`GET https://evo.elvaco.se/api/v1/meters`,

In [40]:
dates = Dates.DateTime(2017, 12), Dates.DateTime(2019, 12)

(2017-12-01T00:00:00, 2019-12-01T00:00:00)

In [41]:
@time meters = Evo.get_meters(access, dates..., 0)

  1.453754 seconds (27.08 k allocations: 1.502 MiB)


Dict{String,Any} with 11 entries:
  "number"           => 0
  "empty"            => false
  "pageable"         => Dict{String,Any}("pageSize"=>50,"unpaged"=>false,"sort"…
  "totalPages"       => 105
  "last"             => false
  "content"          => Any[Dict{String,Any}("address"=>"42545","location"=>Dic…
  "totalElements"    => 5230
  "first"            => true
  "size"             => 50
  "sort"             => Dict{String,Any}("unsorted"=>true,"sorted"=>false,"empt…
  "numberOfElements" => 50

Get content,

In [42]:
@time content = Evo.get_content(access, dates...)
length(content)

 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105,154.902689 seconds (616.14 k allocations: 39.838 MiB, 0.02% gc time)


5230

In [43]:
t = JuliaDB.table(content; pkey = :id)

Table with 5230 rows, 3 columns:
Columns:
#  colname              type
──────────────────────────────
1  id                   Symbol
2  isReported           Bool
3  readIntervalMinutes  Minute

In [44]:
site = (content = t, )
map(length, site)

(content = 5230,)

Save site,

In [ ]:
@time Evo.savesite(secret; site...)

Load site,

In [46]:
@time site = Evo.loadsite(secret, :content)
map(length, site)

  0.053952 seconds (56.19 k allocations: 2.770 MiB)


(content = 5230,)

Specify patterns,

In [47]:
pattern = Dict(
    Symbol("Energy")                => identity,
    Symbol("Flow")                  => identity,
    Symbol("Forward temperature")   => identity,
    Symbol("Power")                 => identity,
    Symbol("Return temperature")    => identity,
    Symbol("Volume")                => identity,
)

Dict{Symbol,typeof(identity)} with 6 entries:
  :Energy                       => identity
  :Volume                       => identity
  Symbol("Return temperature")  => identity
  :Flow                         => identity
  Symbol("Forward temperature") => identity
  :Power                        => identity

Get pattern nodes,

In [48]:
rs = site.content # JuliaDB.rows(Evo.get_pattern_nodes(patterns; site...))
if true
    rs = [(id = Symbol("0016a5f0-ff31-4dda-9465-0aaf841ce2dd"), )] # [rs[Random.rand(1 : end)]]
end
length(rs)

1

Save data,

In [49]:
@time for r in rs
    for sensor_name in keys(pattern)
        p = secret.target
        p = Evo.dbpath(p, "data", r.id; db = sensor_name)
        if !isfile(p)
            try
                Evo.savedata(access, r.id, sensor_name, dates)
                println("$(p): Done")
            catch e
                if typeof(e) <: InterruptException
                    rethrow(e)
                end
                println("$(p): $(e)")
            end
        end
    end
end

db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Energy.db: Unitful.DimensionError(hr kW, 374123.0)
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Volume.db: Unitful.DimensionError(m^3, 7828.49)
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Return temperature.db: Unitful.DimensionError(°C, 35.0)
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Flow.db: Unitful.DimensionError(m^3 hr^-1, 0.189)
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Forward temperature.db: Unitful.DimensionError(°C, 70.0)
db/evo/veab/data/0016a5f0-ff31-4dda-9465-0aaf841ce2dd/Power.db: Unitful.DimensionError(W, 7500.0)
  9.159599 seconds (6.21 M allocations: 346.927 MiB, 1.15% gc time)


Random choice,

In [50]:
r = rs[Random.rand(1 : end)]

(id = Symbol("0016a5f0-ff31-4dda-9465-0aaf841ce2dd"),)

In [51]:
sensor_names = collect(keys(patterns[r.device_id]))

ErrorException: type NamedTuple has no field device_id

In [52]:
sensor_name = sensor_names[Random.rand(1 : end)]

UndefVarError: UndefVarError: sensor_names not defined

Load data,

In [53]:
data = Linckii.loaddata(secret, r.node_id, sensor_name)

UndefVarError: UndefVarError: Linckii not defined

Simple graphics,

In [54]:
@df data plot(:datetime, :value, label = Linckii.dbpath(r.node_id; db = sensor_name))

UndefVarError: UndefVarError: data not defined